## IAMSI -- 2022-2023

# TME 07: Règles d'association


<font color="RED" size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms**


_Double-cliquer ici et insérer les noms et prénoms de votre binôme_


## Présentation


### But de ce TME

Ce TME a pour but de réaliser une implémentation _intuitive_ de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Contenu de l'archive téléchargée

Une fois détarée, l'archive **tme-07.tgz** crée le répertoire `tme07` contenant :

- un répertoire `dataset` contenant des bases d'exemples pour tester les algorithmes
- deux fichiers exécutables (Linux): `apriori32` et `fpgrowth32` (ces 2 programmes ont été développés par Christian Borgelt, plus d'infos sur sa page web https://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance. Si vous n'êtes pas sous Linux, vous pouvez trouver une version pour votre OS ici:
  - `apriori` : https://borgelt.net/apriori.html
  - `fpgrowth` : https://borgelt.net/fpgrowth.html


#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce notebook complété par les réponses aux questions posées.

Ce compte-rendu est à poster :

- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un complément peut être envoyé **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : _Nom1_Nom2.ipynb_

- _Nom1_ et _Nom2_ : noms des membres du binôme
- ne pas compresser ou faire une archive : envoyer le notebook tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les joindre au message.


In [1]:
# Vérification de la version de Python utilisée:
import sys

sys.path  # le path doit contenir python3.5 ou une version supérieure


['/home/aymeric/Documents/DAC/S2/IAMSI/tme07',
 '/home/aymeric/.pyenv/versions/3.10.10/lib/python310.zip',
 '/home/aymeric/.pyenv/versions/3.10.10/lib/python3.10',
 '/home/aymeric/.pyenv/versions/3.10.10/lib/python3.10/lib-dynload',
 '',
 '/home/aymeric/.pyenv/versions/3.10.10/lib/python3.10/site-packages']

In [78]:
from pprint import pprint


## Traitement d'une base d'apprentissage

### Chargement de la base

On commence par travailler sur la base exemple du fichier `exemple-1.txt` (fourni dans le répertoire datasets).

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :


In [2]:
import csv

with open("datasets/exemple-1.txt", "r") as fichier:
    lecteur = csv.reader(fichier, delimiter=" ")
    i = 0
    for ligne in lecteur:
        i += 1
        print("ligne", i, ":", ligne, set(ligne))


ligne 1 : ['a', 'b', 'c'] {'c', 'a', 'b'}
ligne 2 : ['a', 'd', 'e'] {'e', 'a', 'd'}
ligne 3 : ['b', 'c', 'd'] {'b', 'd', 'c'}
ligne 4 : ['a', 'b', 'c', 'd'] {'c', 'a', 'd', 'b'}
ligne 5 : ['b', 'c'] {'b', 'c'}
ligne 6 : ['a', 'b', 'd'] {'a', 'd', 'b'}
ligne 7 : ['d', 'e'] {'e', 'd'}
ligne 8 : ['a', 'b', 'c', 'd'] {'c', 'a', 'd', 'b'}
ligne 9 : ['c', 'd', 'e'] {'e', 'd', 'c'}
ligne 10 : ['a', 'b', 'c'] {'c', 'a', 'b'}


<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `chargeBase` qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des `set()` donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble Python plutôt que comme une liste. Il est ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.


In [25]:
def chargeBase(filepath, delimiteur=" "):
    """Renvoie les itemsets contenus dans un fichier texte.

    Parameters
    ----------
    filename : str
        Chemin vers le fichier.
    delimiteur : str, default=" "
        Le caractère utilisé pour séparer les données.

    Returns
    -------
    base : dict of set
        Ensemble des itemsets.
        Clé : numéro de la transaction (numéro de la ligne).
        Valeur : itemset correspondant.
    """
    import csv

    base = {}
    with open(filepath, "r") as fichier:
        lecteur = csv.reader(fichier, delimiter=delimiteur)
        for row, ligne in enumerate(lecteur):
            base[row] = set(ligne)
    return base


In [4]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase("datasets/exemple-1.txt")
Base1


Résultat du chargement de 'exemple-1.txt', on obtient : 


{0: {'a', 'b', 'c'},
 1: {'a', 'd', 'e'},
 2: {'b', 'c', 'd'},
 3: {'a', 'b', 'c', 'd'},
 4: {'b', 'c'},
 5: {'a', 'b', 'd'},
 6: {'d', 'e'},
 7: {'a', 'b', 'c', 'd'},
 8: {'c', 'd', 'e'},
 9: {'a', 'b', 'c'}}

On utilise la variable **Base1** dans la suite pour faire référence à cette base.


## Itemsets et support


<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `noms_items` qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.


In [5]:
def noms_items(base):
    """Renvoie l'ensemble des items composant une base.

    Parameters
    ----------
    base : dict of set
        Ensemble des itemsets.

    Returns
    -------
    set
        Items uniques de la base.
    """
    return set().union(*[itemset for itemset in base.values()])


In [81]:
print("Pour la BASE précédente :")
noms_items(Base1)


Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `singletons` qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

_Remarque_ : attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [7]:
def singletons(base):
    """Rend la liste des itemsets de taille 1 d'une base.

    Parameters
    ----------
    base : dict of set
        Ensemble des itemsets.

    Returns
    -------
    list of set
        Liste des singletons de la base.
    """
    return [{singleton} for singleton in noms_items(base)]


In [8]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)


Exemple: singletons(Base1) rend :


[{'d'}, {'b'}, {'a'}, {'c'}, {'e'}]

<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `comptage` qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.


In [9]:
def comptage(base, itemset):
    """Renvoie le nombre de transactions contenant un itemset dans une base.

    Parameters
    ----------
    base : dict of set
        Ensemble des itemsets.
    itemset : set
        Itemset.

    Returns
    -------
    int
        Nombre de transactions contenant l'itemset donné.
    """
    return sum([itemset.issubset(base[key]) for key in base])


In [82]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print(
    "comptage(Base1, {'a','b','c'}) rend la valeur %s."
    % comptage(Base1, {"a", "b", "c"})
)
print("comptage(Base1, {'a'}) rend la valeur %s." % comptage(Base1, {"a"}))


Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1, {'a','b','c'}) rend la valeur 4.
comptage(Base1, {'a'}) rend la valeur 6.


<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `support` qui, pour une BASE et un itemset donné, rend le support de cet itemset dans la BASE.


In [11]:
comptage(Base1, {"a", "b", "c"})


def support(base, itemset):
    """Calcule le support d'un itemset d'une base.

    .. math:: \text{support} = \frac {n_{\text{itemset}}} {n}

    Parameters
    ----------
    base : dict of set
        Ensemble des itemsets.
    itemset : set
        Itemset.

    Returns
    -------
    float
        Support de l'itemset donné.
    """
    return comptage(base, itemset) / len(base)


In [12]:
print(
    "Support de l'itemset {'a','b','c'} dans la base précédente : "
    + str(support(Base1, {"a", "b", "c"}))
)
print(
    "Support de l'itemset {'a'} dans la base précédente : " + str(support(Base1, {"a"}))
)


Support de l'itemset {'a','b','c'} dans la base précédente : 0.4
Support de l'itemset {'a'} dans la base précédente : 0.6


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire _datasets_ (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).


<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `apriori_gen` qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.

*Rappel de l'algorithme* :

<font style="font-variant: small-caps">Apriori-Gen</font>($F$)<br>
Entrée : $F$ ensemble d’itemsets fréquents de longueur $k$<br>
$C \leftarrow \{c = f_1 ∪ f_2 \in F \times F$ tels que $|c| = k + 1\}$<br>
pour chaque $c \in C$ faire<br>
&emsp; pour chaque $i \in c$ faire<br>
&emsp;&emsp;si $c \setminus \{i\} \in F$ alors<br>
&emsp;&emsp;&emsp;$C \leftarrow C \setminus \{c\}$<br>
retourner $C$<br>

In [125]:
from itertools import combinations

def apriori_gen(itemsets):
    """Applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats
    à partir d'une liste d'itemsets.

    Parameters
    ----------
    itemsets : list
        Liste d'itemsets de même longueur k.

    Returns
    -------
    list
        Liste des itemsets candidats de longueurs k+1.
    """
    C = []
    k = len(itemsets[0])
    for f1, f2 in combinations(itemsets, 2):
        c = f1.union(f2)
        if len(c) == k + 1 and c not in C:
            C.append(c)
        for i in c:
            if c.difference({i}) not in itemsets and c in C:
                C.remove(c)
    return C
    
%timeit apriori_gen([{"a"}, {"b"}, {"c"}, {"d"}])
%timeit apriori_gen([{"a", "b"}, {"a", "d"}, {"b", "d"}, {"b", "c"}, {"c", "d"}])

4.84 µs ± 11 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
11.2 µs ± 11 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [1]:
def apriori_gen(itemsets):
    """Variante avec deux boucles."""
    C = []
    k = len(itemsets[0])
    for f1 in itemsets:
        for f2 in itemsets:
            c = f1.union(f2)
            if len(c) == k + 1 and c not in C:
                C.append(c)
            for i in c:
                if c.difference({i}) not in itemsets and c in C:
                    C.remove(c)
    return C
    
%timeit apriori_gen([{"a"}, {"b"}, {"c"}, {"d"}])
%timeit apriori_gen([{"a", "b"}, {"a", "d"}, {"b", "d"}, {"b", "c"}, {"c", "d"}])

10.4 µs ± 23.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
24.4 µs ± 245 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [128]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{"a"}, {"b"}, {"c"}, {"d"}]))

print(
    "\nExemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend "
)
print(apriori_gen([{"a", "b"}, {"a", "d"}, {"b", "d"}, {"b", "c"}, {"c", "d"}]))


Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'a', 'b'}, {'a', 'c'}, {'a', 'd'}, {'b', 'c'}, {'b', 'd'}, {'d', 'c'}]

Exemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend 
[{'a', 'd', 'b'}, {'b', 'd', 'c'}]


<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `apriori` qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément et un itemset trouvé et le deuxième élément est la valeur de support correspondante.


In [52]:
def apriori(base, minsup):
    C = [singletons(base)]
    F = []
    k = 0
    while len(C[k]) > 0:
        Fk = []
        for c in C[k]:
            s = support(base, c)
            if s >= minsup:
                Fk.append(c)
                F.append((c, s))
        gen = apriori_gen(Fk)
        C.append(gen)
        k += 1
    return F


# %timeit apriori(Base1, 0.3)


In [54]:
print("Exemple: apriori(Base1, 0.3) rend ")
pprint(apriori(Base1, 0.3))


Exemple: apriori(Base1, 0.3) rend 
[({'d'}, 0.7),
 ({'b'}, 0.7),
 ({'a'}, 0.6),
 ({'c'}, 0.7),
 ({'e'}, 0.3),
 ({'b', 'd'}, 0.4),
 ({'a', 'd'}, 0.4),
 ({'d', 'c'}, 0.4),
 ({'e', 'd'}, 0.3),
 ({'a', 'b'}, 0.5),
 ({'b', 'c'}, 0.6),
 ({'a', 'c'}, 0.4),
 ({'a', 'b', 'd'}, 0.3),
 ({'b', 'd', 'c'}, 0.3),
 ({'a', 'b', 'c'}, 0.4)]


## Expérimentations


### Vérification avec la base du TD

Utiliser la fonction `apriori` avec la base de transactions du TD afin de vérifier que les itemsets obtenus sont corrects.


In [26]:
BaseTD = chargeBase("datasets/td.txt")
BaseTD


{0: {'a', 'b', 'd', 'e'},
 1: {'b', 'c', 'd'},
 2: {'a', 'b', 'd', 'e'},
 3: {'a', 'c', 'd', 'e'},
 4: {'b', 'c', 'd', 'e'},
 5: {'b', 'd', 'e'},
 6: {'c', 'd'},
 7: {'a', 'b', 'c'},
 8: {'a', 'd', 'e'},
 9: {'b', 'd'}}

In [55]:
print("Exemple : apriori(BaseTD, 0.3) rend :")
pprint(apriori(BaseTD, 0.3))


Exemple : apriori(BaseTD, 0.3) rend :
[({'d'}, 0.9),
 ({'b'}, 0.7),
 ({'a'}, 0.5),
 ({'c'}, 0.5),
 ({'e'}, 0.6),
 ({'b', 'd'}, 0.6),
 ({'a', 'd'}, 0.4),
 ({'d', 'c'}, 0.4),
 ({'e', 'd'}, 0.6),
 ({'a', 'b'}, 0.3),
 ({'b', 'c'}, 0.3),
 ({'e', 'b'}, 0.4),
 ({'e', 'a'}, 0.4),
 ({'e', 'b', 'd'}, 0.4),
 ({'e', 'a', 'd'}, 0.4)]


### Comparaisons avec les implémentations apriori et fpgrowth


<font color="RED" size="+1">**[Q]**</font> Tester votre fonction `apriori` sur les 3 bases données dans datasets et confronter les résultats obtenus avec ceux fournis par l'exécutable `apriori`.


In [56]:
print("Résultat du chargement de 'mushrooms.txt', on obtient : ")
Base_mushrooms = chargeBase("datasets/mushrooms.txt", delimiteur=",")
# Base_mushrooms
print(len(Base_mushrooms), " transactions.")


Résultat du chargement de 'mushrooms.txt', on obtient : 
8416  transactions.


In [57]:
print("Pour la BASE précédente :")
noms_items(Base_mushrooms)
print("Exemple: singletons(Base_mushrooms) rend :")
singletons(Base_mushrooms)
print("Nombre de singletons :", len(singletons(Base_mushrooms)))


Pour la BASE précédente :
Exemple: singletons(Base_mushrooms) rend :
Nombre de singletons : 70


In [59]:
pprint(apriori(Base_mushrooms, 0.9))


[({'WHITE'}, 0.9771863117870723),
 ({'PARTIAL'}, 1.0),
 ({'FREE'}, 0.9743346007604563),
 ({'ONE'}, 0.9230038022813688),
 ({'WHITE', 'PARTIAL'}, 0.9771863117870723),
 ({'FREE', 'WHITE'}, 0.9743346007604563),
 ({'WHITE', 'ONE'}, 0.9001901140684411),
 ({'FREE', 'PARTIAL'}, 0.9743346007604563),
 ({'ONE', 'PARTIAL'}, 0.9230038022813688),
 ({'FREE', 'ONE'}, 0.9001901140684411),
 ({'FREE', 'WHITE', 'PARTIAL'}, 0.9743346007604563),
 ({'WHITE', 'ONE', 'PARTIAL'}, 0.9001901140684411),
 ({'FREE', 'WHITE', 'ONE'}, 0.9001901140684411),
 ({'FREE', 'ONE', 'PARTIAL'}, 0.9001901140684411),
 ({'FREE', 'ONE', 'PARTIAL', 'WHITE'}, 0.9001901140684411)]


In [60]:
print("Résultat du chargement de 'titanic-red.csv', on obtient : ")
Base_titanic = chargeBase("datasets/titanic-red.csv", delimiteur=",")
print(len(Base_titanic), " transactions.")


Résultat du chargement de 'titanic-red.csv', on obtient : 
741  transactions.


In [62]:
print("Pour la BASE Titanic :")
noms_items(Base_titanic)
print("Exemple: singletons(Base_titanic) rend :")
singletons(Base_titanic)


Pour la BASE Titanic :
Exemple: singletons(Base_titanic) rend :


[{'EnglandNewYorkNY'},
 {'ProvidenceRI'},
 {'ZurichSwitzerland'},
 {'BelmontMA'},
 {'SurbitonHillSurrey'},
 {'YoungstownOH'},
 {'SwedenChicagoIL'},
 {'NewYorkNYStamfordCT'},
 {'WimbledonParkLondonHaylingIslandHants'},
 {'EnglandSacramentoCA'},
 {'PenzanceCornwallAkronOH'},
 {'CoAthloneIrelandNewYorkNY'},
 {'BergenNorway'},
 {"StAnne'sonSeaLancashire"},
 {'BroomfieldChelmsfordEngland'},
 {'HornseyEngland'},
 {'BuffaloNY'},
 {'WestHampsteadLondonNeepawaMB'},
 {'BarcelonaSpainHavanaCuba'},
 {'GuernseyMontclairNJand/orToledoOhio'},
 {'WorcesterEngland'},
 {'HarrisburgPA'},
 {'SwindonEngland'},
 {'SydneyAustralia'},
 {'MamaroneckNY'},
 {'RuotsinphyhtaaFinlandNewYorkNY'},
 {'ElkinsParkPA'},
 {'NewYorkNYGreenwichCT'},
 {'KontiolahtiFinlandDetroitMI'},
 {'TokyoJapan'},
 {'MilfordNH'},
 {'LimaPeru'},
 {'DowagiacMI'},
 {'LosAngelesCA'},
 {'GreenportNY'},
 {'GunturIndiaBentonHarbourMI'},
 {'BrennesNorwayNewYork'},
 {'HaverfordPACooperstownNY'},
 {'SkaraSwedenRockfordIL'},
 {'GreenBayWI'},
 {'Guer

In [66]:
Lres = apriori(Base_titanic, 0.2)

# tri sur les valeurs de supports:
# print(sorted(Lres, key=lambda data: data[1]))

# tri sur la taille des itemsets
# print(sorted(Lres, key=lambda data: len(data[0])))

for t in sorted(Lres, key=lambda data: data[1], reverse=True):
    if len(t[0]) > 1:
        print(t)


({'male', 'dead'}, 0.46963562753036436)
({'male', 'Southampton'}, 0.46288798920377866)
({'dead', 'Southampton'}, 0.41295546558704455)
({'male', 'dead', 'Southampton'}, 0.3684210526315789)
({'alive', 'female'}, 0.32388663967611336)
({'2nd', 'Southampton'}, 0.30634278002699056)
({'alive', 'Southampton'}, 0.2982456140350877)
({'female', 'Southampton'}, 0.2483130904183536)
({'1st', 'alive'}, 0.23076923076923078)
({'male', '1st'}, 0.21997300944669365)
({'1st', 'Southampton'}, 0.20917678812415655)
({'male', '2nd'}, 0.20647773279352227)
({'alive', 'female', 'Southampton'}, 0.203778677462888)


<font color="RED" size="+1">**[Q]**</font> Utiliser l'éxécutable `apriori` pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance).


In [77]:
!./apriori -s30 datasets/exemple-1.txt out/exemple-1.out
!cat out/exemple-1.out

./apriori - find frequent item sets with the apriori algorithm
version 6.31 (2022.11.22)        (c) 1996-2022   Christian Borgelt
reading datasets/exemple-1.txt ... [5 item(s), 10 transaction(s)] done [0.00s].
filtering, sorting and recoding items ... [5 item(s)] done [0.00s].
sorting and reducing transactions ... [8/10 transaction(s)] done [0.00s].
building transaction tree ... [12 node(s)] done [0.00s].
checking subsets of size 1 2 3 done [0.00s].
writing out/exemple-1.out ... [15 set(s)] done [0.00s].
e d (30)
e (30)
a (60)
a d (40)
a d b (30)
a c b (40)
a c (40)
a b (50)
d (70)
d c (40)
d c b (30)
d b (40)
c (70)
c b (60)
b (70)


<font color="RED" size="+1">**[Q]**</font> Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


<font color="RED" size="+1">**[Q]**</font> Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.

Par exemple:

\begin{align}
\text{Interest}(X \longrightarrow Y) & = \frac{P(X,Y)}{P(X)}P(Y) \nonumber\\
\text{IS}(X \longrightarrow Y) & = \frac{P(X,Y)}{\sqrt{P(X)P(Y)}} \nonumber
\end{align}


### Utilisation de fp-growth


<font color="RED" size="+1">**[Q]**</font> Utiliser `fpgrowth` pour générer des règles d'association sur les différentes
bases de données. Comparer avec les résultats obtenus dans la section précédente. En particulier, comparer les temps d'éxecution.


<font color="RED" size="+1">**[Q]**</font> En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.


## Annexes : utilisation des exécutables _apriori_ et _fpgrowth_

La documentation complète de ces programmes est disponible aux URL suivantes:

- pour _apriori_: https://www.borgelt.net/doc/apriori/apriori.html
- pour _fp-growth_: https://www.borgelt.net/doc/fpgrowth/fpgrowth.html

Leur format d'utilisation général est:

        ./<programme> [options] infile [outfile]

(où <`programme`> est soit _apriori_, soit _fpgrowth_).

Les options de base de ces 2 programmes sont:

- sans argument: génération des itemsets fréquents (argument {`-ts` activé par défaut)
- _tr_: pour obtenir les règles d'associations
- _s_: pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- _m_: pour fournir un nombre minimum d'items dans un itemset
- _n_: pour fournir un nombre maximum d'items dans un itemset

Par exemple (commandes lancées dans le répertoire père du répertoire _datasets/_):

        ./apriori -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out
        ./fpgrowth -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out
